# 오버피팅 

- 매개변수가 많고 표현력이 높은 모델

- 훈련데이터가 적은 모델

에서 발생

In [3]:
#데이터를 300개만 사용하고 7개의 층을 사용해 고의적으로 오버피팅함
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test,t_test) = load_mnist(normalize = True)
x_train = x_train[:300]
t_train = t_train[:300]

In [4]:
network = MultiLayerNet(input_size = 784, hidden_size_list = [100,100.,100,100,100,100,], output_size = 10)
optimizer = SGD(lr=0.01)

max_epochs = 201
train_size = x_train.shape[0]
batch_size = 100

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/batch_size,1)
epoch_cnt = 0

for i in range(1000000000):
    batch_mask = np.random.choice(train_size,batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grads = network.gradient(x_batch, t_batch)
    optimizer.update(network.params, grads)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        
        epoch_cnt += 1
        if epoch_cnt >=max_epochs:
            break

NameError: name 'MultiLayerNet' is not defined

### 오버피팅 억제 방법 : 가중치 감소

오버피팅은 가중치의 매개변수 값이 커서 발생하는 경우가 많다 --> 가중치 감소를 통해서 해결

가중치 감소 : 손실함수에 (1/2) * 람다 * W^2를 더한다.
    
가중치의 기울기를 구하는 계산에서는 오차역전파법에 따른 결과에 람다 * W를 더한다.

### 오버피팅 억제 방법 : 드롭아웃

신경망이 복잡해진다면 가중치 감소만으로는 대응하기 어려움 --> 드롭아웃 사용

In [5]:
class Dropout:
    def __init__(self,dropout_ratio = 0.5):  
        self.dropout_ratio = dropout_ratio
        self.mask = None
    
    def forward(self,x,train_flg = True):
        if train_flg:
            self.mask = np.random_rand(*x.shape) > self.dropout_ratio  #np.random_rand : 0~1사이의 난수 생성
            return x * self.mask
        else:
            return x * (1.0 - self.dropout_ratio)
        
    def backward(self,dout):
        return dout * self.mask

### 오버피팅 억제 방법 : 앙상블

--> 개별적으로 학습시킨 여러 모델의 출력을 평균내어 추론하는 방식